CODE BLOCK TO EXTRACT WEATHER FROM LAT LONG

In [20]:
import pandas as pd
import reverse_geocode as rg
import numpy as np

In [21]:
from datetime import datetime,timedelta
from astropy.time import Time
from meteostat import Point, Hourly

In [22]:
LIMITED = False
METEOSTAT = False

In [23]:
icouk_data_file = '..\\Data\\icouk_sighting_data_with_params.csv'
#icop_data_file = '..\\Data\\icop_ahmed_2020_sighting_data_with_params.csv'
#alrefay_data_file = '..\\Data\\alrefay_2018_sighting_data_with_params.csv'
#allawi_data_file = '..\\Data\\schaefer_odeh_allawi_2022_sighting_data_with_params.csv'
#yallop_data_file = '..\\Data\\yallop_sighting_data_with_params.csv'

#icouk_data = pd.read_csv(icouk_data_file)
#icop_data = pd.read_csv(icop_data_file)
#alrefay_data = pd.read_csv(alrefay_data_file)
#allawi_data = pd.read_csv(allawi_data_file)
#yallop_data = pd.read_csv(yallop_data_file)


#data = pd.concat([icouk_data,icop_data,alrefay_data,yallop_data])
data = pd.read_csv(icouk_data_file)


#data = data[['Latitude', "Longitude", 'Date']]
if LIMITED:
    df = data.copy('Deep').head(20)
else:
    df = data.copy('Deep')

In [24]:
#index used
i = 500

In [25]:
#get city from lat long, not rlly needed
# city_arr = []
# city_dic = rg.search(df[['Latitude', 'Longitude']])
# for i in range(len(city_dic)):
#     city_arr.append((city_dic[i])['city']+','+(city_dic[i])['country_code'])

# df['City'] = city_arr

In [26]:
#df

In [27]:
if METEOSTAT:
    # adds meteostat weather data for dataset

    weather_data =[]
    for i in range(len(city_dic)):
        day = Time(df['Date'][i],format='jd').to_datetime()
        weather_data_temp = Hourly(Point(df['Latitude'][i],df['Longitude'][i]), day-timedelta(hours=0.5), day + timedelta(hours=0.5)) #day+timedelta(days=1))
        x = weather_data_temp.fetch()
        if (len(x)>0):
            weather_data.append(x)
        else:
            weather_data.append( pd.DataFrame(np.zeros([1, 11])*np.nan, columns = ['temp', 'dwpt', 'rhum', 'prcp', 'snow','wdir ','wspd','wpgt','pres', 'tsun', 'coco']))
    weather_dataset = (pd.concat(weather_data, ignore_index=True)).drop(['wdir','wspd', 'wpgt'], axis=1)

In [28]:
import requests
from bs4 import BeautifulSoup
import re


In [29]:
best_time = Time(df['Date'][i], format='jd') #best time in julian
DATE = str(int(np.round(best_time.unix/60/30)*60*30)) #convert julian date to UNIX timestamp HOWEVER has british summertime
print(DATE)
url = 'https://www.weatheronline.co.uk/weather/maps/current?LANG=en&DATE='+ DATE  +'&CONT=ukuk&LAND=UK&KEY=UK&SORT=4&UD=0&INT=06&TYP=bedeckung&ART=tabelle&RUBRIK=akt&R=310&CEL=C&SI=mph'
response = requests.get(url)
print(response.status_code)

1646330400


200


In [30]:
soup = BeautifulSoup(response.text, 'html.parser')

In [31]:
table = soup.find('table', class_ = 'gr3')

table = str(table)
#print(table)

In [32]:
names = re.findall("(?<=CEL=C\">)(.*?)(?= \()", table)
code = re.findall("(?<=WMO=)(.*?)(?=\&amp)", table)
cloud_level = re.findall("(?<=<td>)(\d)(?=<\/td>)", table)
print(names)
print(code)
print(cloud_level)

['Yeovil', 'Weybourne', 'Capel Curig', 'Altnaharra', 'Aboyne', 'Tulloch Bridge', 'Drumalbin', 'Inverbervie', 'Strathallan', 'Dundrennan', 'MoD Boscombe Down', 'Lough Fea', 'RAF Lyneham', 'Exeter Airport No.', 'RAF Brize Norton', 'Herstmonceux', 'Hawarden', 'Aviemore', 'RAF Odiham', 'Lerwick', 'Lake Vyrnwy', 'Thorncliffe', 'Shobdon', 'Cranwell', 'Albemarle', 'Edinburgh Gogarbank', 'Waddington', 'Warcop Range', 'Keswick', 'Redesdale Camp', 'Bingley', 'Crosby', 'Carlisle', 'Credenhill', 'Pershore', 'Church Lawford', 'RAF Little Rissington', 'Bedford', 'Nottingham/Watnall', 'Larkhill', 'Charlwood', 'Coleshill', 'Guernsey Airport', 'Isle of Man Airport', 'Coningsby', 'Lossiemouth', 'Castlederg', 'Benson', 'Northolt', 'Loch Glascarnoch', 'Plymouth MtBatten', 'Camborne', 'Wattisham Airport', 'Glenanne', 'Marham', 'Manston', 'Wainfleet All Saints', 'Portglenone', 'Magilligan']
['03853', '03488', '03305', '03044', '03080', '03047', '03155', '03088', '03144', '03153', '03746', '03911', '03740', 

METEOSTAT distance comparison

In [33]:
from meteostat import Stations

In [34]:
day = best_time.to_datetime()
print(day)

2022-03-03 18:08:28.896013


In [35]:
stations = Stations().nearby(df['Latitude'][i],df['Longitude'][i])
stations = stations.inventory('hourly', day)
station = stations.fetch(10)

In [36]:
station

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,distance
id,,,,,,,,,,,,,,,,
03781,Kenley,GB,ENG,03781,<NA>,51.3000,-0.0833,170.0,Europe/London,2018-01-27,2023-11-17,2018-01-28,2022-04-23,2018-01-01,2022-01-01,9338.200596
EGKB0,Biggin Hill / berry's green,GB,ENG,<NA>,EGKB,51.3167,0.0333,183.0,Europe/London,1988-01-05,2023-11-17,2020-01-14,2022-04-26,2020-01-01,2022-01-01,11857.143928
EGLC0,London / Abbey Wood,GB,ENG,<NA>,EGLC,51.5000,0.1167,5.0,Europe/London,1988-01-29,2023-11-17,2007-09-26,2022-04-27,2016-01-01,2022-01-01,19846.395835
03776,London Gatwick Airport,GB,ENG,03776,EGKK,51.1500,-0.1833,62.0,Europe/London,1973-01-01,2023-11-17,1973-01-02,2022-04-25,1961-01-01,2022-01-01,26584.522674
03772,London Heathrow Airport,GB,ENG,03772,EGLL,51.4833,-0.4500,24.0,Europe/London,1948-12-01,2023-11-17,1948-12-01,2023-11-11,1948-01-01,2022-01-01,26687.835641
03672,Northolt,GB,ENG,03672,EGWU,51.5500,-0.4167,38.0,Europe/London,1973-01-01,2023-11-17,1973-01-05,2022-04-25,2005-01-01,2022-01-01,28718.403901
03763,Bracknell / Beaufort Park,GB,ENG,03763,EGRR,51.3833,-0.7833,74.0,Europe/London,1973-01-01,2023-01-31,1992-03-30,2003-09-19,NaT,NaT,47419.174043
03768,Farnborough,GB,ENG,03768,EGLF,51.2833,-0.7667,65.0,Europe/London,1988-05-01,2023-11-17,1988-05-02,2022-04-25,2005-01-01,2022-01-01,47634.495131
03673,London / Chiltern Green / Newmill End / Stopsl...,GB,ENG,03673,EGGW,51.8667,-0.3667,160.0,Europe/London,1988-01-05,2023-11-17,1988-01-31,2022-04-25,2009-01-01,2022-01-01,56813.820979


In [37]:
#finds the first agreement of metrostat and weather online to give closest station
closest_name = station['name'][np.where(np.in1d(station.index, code))[0][0]]
print(closest_name)
cloud_level[np.where(np.in1d(names, closest_name))[0][0]]

Northolt


'7'